In [1]:
!pip install pydantic-ai==0.1.3

  Obtaining dependency information for pydantic-ai==0.1.3 from https://files.pythonhosted.org/packages/c7/59/bcef9d0328a0f5e262c9584e53f488cd2a64ecacf03732ae01152a6ec264/pydantic_ai-0.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic-ai-slim[anthropic,bedrock,cli,cohere,evals,groq,mcp,mistral,openai,vertexai]==0.1.3 from https://files.pythonhosted.org/packages/c0/bd/823ba9aff48490bd51ec5aaf760307a6d7f695489f5f162b1dbeaad6adb9/pydantic_ai_slim-0.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for eval-type-backport>=0.2.0 from https://files.pythonhosted.org/packages/ce/31/55cd413eaccd39125368be33c46de24a1f639f2e12349b0361b4678f3915/eval_type_backport-0.2.2-py3-none-any.whl.metadata
  Obtaining dependency information for griffe>=1.3.2 from https://files.pythonhosted.org/packages/b1/5e/38b408f41064c9fcdbb0ea27c1bd13a1c8657c4846e04dab9f5ea770602c/griffe-1.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for httpx>=0.27 from https

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
aiobotocore 2.5.0 requires botocore<1.29.77,>=1.29.76, but you have botocore 1.37.37 which is incompatible.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.11.3 which is incompatible.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0.2 which is incompatible.
conda-repo-cli 1.0.75 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2025.3.2 which is incompatible.
transformers 4.32.1 requires tokenizers!=0.11.3,<0.14,>=0.11.1, bu

In [24]:
import nest_asyncio
nest_asyncio.apply()

In [25]:
import requests
BASE_URL = 'https://api.openweathermap.org/data/2.5/weather'
API_KEY = '842b3e131a153cb327f338e9604228cc'

# weather forecast function
def find_weather(city:str)->dict:
    """This function returns the current weather forecast for the given city"""
    units="metric"
    params={ 
        'q': city,
        'appid': API_KEY,
        'units': units
    }
    
    response =requests.get(BASE_URL, params=params)
    result=response.json()
    return result

In [26]:
op=find_weather("chennai")
print(op)

{'coord': {'lon': 80.2785, 'lat': 13.0878}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'base': 'stations', 'main': {'temp': 34.49, 'feels_like': 41.49, 'temp_min': 33.34, 'temp_max': 36.16, 'pressure': 1004, 'humidity': 71, 'sea_level': 1004, 'grnd_level': 1003}, 'visibility': 6000, 'wind': {'speed': 6.17, 'deg': 140}, 'clouds': {'all': 20}, 'dt': 1745057513, 'sys': {'type': 2, 'id': 2093220, 'country': 'IN', 'sunrise': 1745022230, 'sunset': 1745067109}, 'timezone': 19800, 'id': 1264527, 'name': 'Chennai', 'cod': 200}


## Pydantic Agents

In [27]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [28]:
 os.environ["GROQ_API_kEY"]="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"

In [42]:
# Define the output

class WeatherForecast(BaseModel):
    location: str
    description: str
    temperature_celsius: float
    pressure: int

In [43]:
weather_agent= Agent(
    model="groq:llama-3.1-8b-instant",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt = ("You are a helpful weather assistant. Use the 'get_weather_forecast' tool to find current weather "
        "conditions for any city. Provide clean and friendly answers.")
    
)


In [44]:
# weather forecast tool
@weather_agent.tool

def get_weather_forecast(ctx: RunContext, city: str) -> WeatherForecast:
    """
    Returns current weather forecast for the city using OpenweatherMap API:
    """
    url = 'https://api.openweathermap.org/data/2.5/weather'
    api_key = '842b3e131a153cb327f338e9604228cc'
    params={ 
        'q': city,
        'appid': API_KEY,
        'units': 'metric'
    }
    
    res=requests.get(url, params=params).json()
    
    return WeatherForecast(
        location=res['name'],
        description=res['weather'][0]['description'].capitalize(),
        temperature_celsius=res['main']['temp'],
        pressure=res['main']['pressure']
    )

In [45]:
question=input("Ask about the weather: ")
result=weather_agent.run_sync(question)
print('\n forecast: ', result.output)

Ask about the weather: Galway

 forecast:  It's currently light rain in Galway with a temperature of 9.92°C and a pressure of 1007.
